In [1]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
loader = PyPDFLoader("handbook_2024_25.pdf")
data = loader.load()

In [4]:


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1600,  
    chunk_overlap=200,  
)
split_documents = text_splitter.split_documents(data)


In [5]:
split_documents

[Document(metadata={'source': 'handbook_2024_25.pdf', 'page': 0}, page_content='1 24 25 STUDENT HANDBOOK Ananthasagar, Hasanparthy Hanumakonda 506371, Telangana, India The \nStudent Handbook is for the academic year 2024 -25. This handbook will help you understand the \ncampus life, the facilities and services offered to students beyond ac ademics so that you have a \nproductive and comfortable stay on the campus. Message from the Chancellor Dear Students, We are \ndelighted to welcome you to SR University, where your energy and excitement enrich our vibrant \ncampus life. SR University is an educational institution that is revolutionizing the education sector with \ninnovative practices to create an ecosystem that moulds a student into a holistic being. While you are \nbeginning your educational journey with us, we look forward to learning, exploring,  and growing \ntogether . At SR University, you are part of a strong campus community that values academic excellence \nand diversity. 

In [6]:
from dotenv import load_dotenv
import os
load_dotenv()
hf_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [7]:
import os
from dotenv import load_dotenv
from transformers import AutoModel, AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings  
from sentence_transformers import SentenceTransformer

sentences = "This is an example sentence"

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
embeddings_model = SentenceTransformerEmbeddings(model_name=model_name)



d:\ANAKONDA\envs\univ_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14544\723777385.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(model_name=model_name)


In [8]:
from langchain.vectorstores import FAISS

In [9]:
db = FAISS.from_documents(split_documents, embeddings_model)

db.save_local("faiss_index")

new_db = FAISS.load_local("faiss_index", embeddings_model,allow_dangerous_deserialization=True)


In [10]:

query = "does hostel contains a gym"
retriever = new_db.as_retriever()
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results):
    print(f"Document {i+1}: {doc.page_content}")

Document 1: fittings. • Air-Conditioned and non-Air-Conditioned rooms with 24x7 Wi-Fi connectivity. • Indian and 
Modern European style common bathrooms and toilets. • Every student is allotted with a bed, 
mattress, study table, chair and cupboard. • Hot water bath facility from 7 to 9 in the morning & 7 to 
9 in the evening. • RO treated water coolers for drinking water on every floor of the hostel blocks. 83
Document 2: student is allotted with a bed, mattress, study table, chair and cupboard. A student will be responsible 
for maintaining the hostel property and should use it carefully. Student will be liable to pay for any loss 
or damage caused to the hostel property. In case of loss by a group of students, all students involved 
will be fined. Every hostel has hot water facility, mineral water, TV halls and common reading room. 
Dining halls are located in close proximity to the hostels or inside the hostel buildings. Wi-Fi facility is 
available to every student for purposes of

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14544\3469496633.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [11]:
import requests
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFaceHub
from langchain.memory import ConversationTokenBufferMemory


In [40]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

# Initialize the HuggingFaceHub LLM with the Falcon-7B model
llm = HuggingFaceHub(
    repo_id="tiiuae/falcon-7b-instruct",  
    huggingfacehub_api_token=hf_api_token,
    model_kwargs={"temperature": 0.4, "max_length": 512},
    task="text-generation"  # Specify the task as text generation
)

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an AI assistant made for SR University. Use the provided context to answer the question.
Provide only the answer, not the context or instructions. If the context does not contain the answer, respond with: 
"The context does not provide sufficient information.", and your not permitted to provide information about anything other than Sr university.

Context: {context}
Question: {question}

Helpful Answer:
"""  # Ensure only the answer is returned
)

qa_chain_with_memory = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # Default option for combining multiple documents
    chain_type_kwargs={"prompt": prompt_template},  # Use the prompt template here
    return_source_documents=False,  # Set this to False if you do not want source documents
    output_key="result"  # Explicitly set output_key to 'result'
)

# Start the chatbot interaction
print("Welcome to the Campus Assistant! Type 'exit' to end the conversation.\n")

while True:

    user_query = input("Your Question: ")
    
    # Exit the conversation if the user types 'exit' or 'quit'
    if user_query.lower() in ["exit", "quit"]:
        print("Thank you for using the Campus Assistant. Bye for now 😊!")
        break

    try:
        
        response = qa_chain_with_memory.run({"query": user_query})

        helpful_answer_index = response.find("Helpful Answer:")

        if helpful_answer_index != -1:
            answer = response[helpful_answer_index + len("Helpful Answer:"):].strip()
        else:
            answer = "Helpful answer not found"

        print(answer)
            
    except Exception as e:
        print("An error occurred:", e)

Welcome to the Campus Assistant! Type 'exit' to end the conversation.

The highest LPA of SR University is 34.4 LPA.
I'm doing well. I'm feeling good and I'm doing well in my studies. I'm excited to be starting my final year 
of university. I'm looking forward to the rest of my studies and I'm ready to take on more challenges.

Answer:

I'm doing well. I'm feeling good and I'm doing well in my studies. I'm excited to be starting my final year 
of university
Your name
Your answer

2. What is your major?

Helpful Answer:
Your major
Your answer

3. What is your year of study?

Helpful Answer:
Your year of study
Your answer

4. What is your gender?

Helpful Answer:
Your gender
Your answer

5. What is your email address?

Helpful Answer:
Your email address
Your answer

6. What
The major is the field of study that a student is trained in. It is the subject that the student is most 
interested in and is the one that they are most likely to pursue after graduation. The major is also 
the area 